In [1]:
import tensorflow as tf
from veggideas.transfer import load_non_trainable_model
from keras import regularizers, layers
from keras import models
from keras import optimizers
from veggideas.load_data import load_train_data, load_val_data

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading test data
Testing data successfully loaded ✅
Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [2]:
train_data = load_train_data()
val_data = load_val_data()
model = load_non_trainable_model()

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅


In [3]:
small_val_data = val_data.take(5)
small_train_data = train_data.take(5)

In [12]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    conv2D_256 = layers.Conv2D(512, 3, padding='same', activation='relu')
    conv2D_512 = layers.Conv2D(512, 3, padding='same', activation='relu')
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    
    # Bounding Box Layer
    def bounding_box_layer(inputs):
        # Assuming the input shape is (batch_size, height, width, channels)
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        num_channels = input_shape[3]
        
        # Define the bounding box coordinates
        x_min = 0.2
        y_min = 0.3
        x_max = 0.8
        y_max = 0.7
        
        # Create a bounding box tensor
        bounding_box = tf.constant([[y_min, x_min, y_max, x_max]])
        
        # Repeat the bounding box for each image in the batch
        bounding_boxes = tf.tile(bounding_box, [batch_size, 1])
        
        # Apply crop and resize to extract features within the bounding box
        roi_pooled = tf.image.crop_and_resize(inputs, bounding_boxes, [0], (7, 7))
        
        return roi_pooled
    
    
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        conv2D_256,
        maxpool_layer,
        conv2D_512,
        maxpool_layer,
        bounding_box_layer,
        layers.Flatten(),
        layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
        layers.Dense(128, activation='relu', bias_regularizer=regularizers.L2(0.001)),
        layers.Dense(64, activation='relu'),
        layers.Dense(15, activation='softmax')
    ])

    opt = optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)


ValueError: Attempt to convert a value (<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>) to a Tensor.

In [5]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    conv2D_256 = layers.Conv2D(512, 3, padding='same', activation='relu')
    conv2D_512 = layers.Conv2D(512, 3, padding='same', activation='relu')
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    
    # Bounding Box Layer
    def bounding_box_layer(inputs):
        # Assuming the input shape is (batch_size, height, width, channels)
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        num_channels = input_shape[3]
        
        # Define the bounding box coordinates
        x_min = 0.2
        y_min = 0.3
        x_max = 0.8
        y_max = 0.7
        
        # Create a bounding box tensor
        bounding_box = tf.constant([[y_min, x_min, y_max, x_max]])
        
        # Repeat the bounding box for each image in the batch
        bounding_boxes = tf.tile(bounding_box, [batch_size, 1])
        
        # Apply crop and resize to extract features within the bounding box
        roi_pooled = tf.image.crop_and_resize(inputs, bounding_boxes, [0], (7, 7))
        
        return roi_pooled
    
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        conv2D_256,
        maxpool_layer,
        conv2D_512,
        maxpool_layer,
        layers.Lambda(bounding_box_layer),  # Wrap the bounding_box_layer in a Lambda layer
        layers.Flatten(),
        layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
        layers.Dense(128, activation='relu', bias_regularizer=regularizers.L2(0.001)),
        layers.Dense(64, activation='relu'),
        layers.Dense(15, activation='softmax')
    ])

    opt = optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)


Epoch 1/10


2023-06-08 10:50:40.413116: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-08 10:50:43.056315: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at crop_and_resize_op.cc:153 : INVALID_ARGUMENT: box_index has incompatible shape


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_5/lambda/CropAndResize' defined at (most recent call last):
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/lieselvranckx/.pyenv/versions/

In [6]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    conv2D_256 = layers.Conv2D(512, 3, padding='same', activation='relu')
    conv2D_512 = layers.Conv2D(512, 3, padding='same', activation='relu')
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    
    # Bounding Box Layer
    def bounding_box_layer(inputs):
        # Assuming the input shape is (batch_size, height, width, channels)
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        num_channels = input_shape[3]
        
        # Define the bounding box coordinates
        x_min = 0.2
        y_min = 0.3
        x_max = 0.8
        y_max = 0.7
        
        # Create a bounding box tensor
        bounding_box = tf.constant([[y_min, x_min, y_max, x_max]])
        
        # Repeat the bounding box for each image in the batch
        bounding_boxes = tf.tile(bounding_box, [batch_size, 1])
        
        # Reshape the box_index to match the expected shape
        box_index = tf.reshape(tf.range(batch_size), (-1, 1))
        
        # Apply crop and resize to extract features within the bounding box
        roi_pooled = tf.image.crop_and_resize(inputs, bounding_boxes, box_index, (7, 7))
        
        return roi_pooled
    
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        conv2D_256,
        maxpool_layer,
        conv2D_512,
        maxpool_layer,
        layers.Lambda(bounding_box_layer),  # Wrap the bounding_box_layer in a Lambda layer
        layers.Flatten(),
        layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
        layers.Dense(128, activation='relu', bias_regularizer=regularizers.L2(0.001)),
        layers.Dense(64, activation='relu'),
        layers.Dense(15, activation='softmax')
    ])

    opt = optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)


Epoch 1/10


ValueError: in user code:

    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
   

In [9]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.2),
        ])

    base_model = model
    conv2D_256 = layers.Conv2D(512, 3, padding='same', activation='relu')
    conv2D_512 = layers.Conv2D(512, 3, padding='same', activation='relu')
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    
    # Bounding Box Layer
    def bounding_box_layer(inputs):
        # Get the height and width from inputs
        height = tf.shape(inputs)[1]
        width = tf.shape(inputs)[2]
        
        # Reshape the inputs to (batch_size, height, width, channels)
        inputs = tf.reshape(inputs, (-1, height, width, 512))
        
        # Assuming the input shape is (batch_size, height, width, channels)
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        num_channels = input_shape[3]
        
        # Define the bounding box coordinates
        x_min = 0.2
        y_min = 0.3
        x_max = 0.8
        y_max = 0.7
        
        # Create a bounding box tensor
        bounding_box = tf.constant([[y_min, x_min, y_max, x_max]])
        
        # Expand dimensions of the bounding box tensor
        bounding_box = tf.expand_dims(bounding_box, axis=0)
        
        # Repeat the bounding box for each image in the batch
        bounding_boxes = tf.tile(bounding_box, [batch_size, 1, 1])
        
        # Reshape the box_index to match the expected shape
        box_index = tf.reshape(tf.range(batch_size), (-1, 1))
        
        # Apply crop and resize to extract features within the bounding box
        roi_pooled = tf.image.crop_and_resize(inputs, bounding_boxes, box_index, (7, 7))
        
        return roi_pooled
    
    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        conv2D_256,
        maxpool_layer,
        conv2D_512,
        maxpool_layer,
        layers.Lambda(bounding_box_layer),  # Wrap the bounding_box_layer in a Lambda layer
        layers.Flatten(),
        layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
        layers.Dense(128, activation='relu', bias_regularizer=regularizers.L2(0.001)),
        layers.Dense(64, activation='relu'),
        layers.Dense(15, activation='softmax')
    ])

    opt = optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(small_train_data, batch_size=32, epochs=10, validation_data=small_val_data)

Epoch 1/10


ValueError: in user code:

    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/Users/lieselvranckx/.pyenv/versions/3.10.6/envs/veggideas/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
   